<div id="image">
<img src="https://www.imt-atlantique.fr/sites/default/files/logo_mt_0_0.png" WIDTH=280 HEIGHT=280>
</div>
<div id="subject">
<CENTER>
</br>


<font size="5"></br> UE Introduction au Machine Learning: Project ML </font></br></div>
</CENTER>
<CENTER>
<font size="3"></br></font></br></div>
</CENTER>
<CENTER>
<span style="color:blue"></span>
</CENTER>

The objective of the project is to apply a Machine Learning model onto two different datasets:
- Banknote Authentication Dataset
- Chronic Kidney Disease:

Workflow : 

1. Import the dataset
2. Clean the data, perform pre-processing
I Replace missing values by average or median values
I Center and normalize the data
3. Split the dataset
I Split between training set and test set
I Split the training set for cross-validation
4. Train the model (including feature selection)
5. Validate the model
Objective: collaboratively implement this workflow and apply it to different ML problems/datasets


# Import Data 



In [231]:
import numpy as np
import scipy.io as sio
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from pylab import *


Informations sur l'ensemble des données :

Les données ont été extraites d'images prises sur des spécimens de billets de banque authentiques et contrefaits. Pour la numérisation, une caméra industrielle habituellement utilisée pour l'inspection des imprimés a été utilisée. Les images finales ont 400x 400 pixels. En raison de l'objectif de l'objet et de la distance de l'objet étudié, des images en échelle de gris avec une résolution d'environ 660 dpi ont été obtenues. L'outil de transformation en ondelettes a été utilisé pour extraire les caractéristiques des images. 


In [244]:
data = pd.read_csv('C:/Users/cleme/OneDrive/Documents/MCE - Machine Learning/Project ML/data_banknote_authentication.txt', sep=",", header=None)
data.columns = ["Variance of WTI","Skewness of WTI","Curtosis of WTI","Entropy of image","Class"]
data.describe()

#1. variance de l'image transformée en ondelettes
#2. asymétrie de l'image transformée en ondelettes
#3. applatissement de l'image transformée en ondelettes 
# 4. entropie de l'image 
# 5. classe (nombre entier)  

,Variance of WTI,Skewness of WTI,Curtosis of WTI,Entropy of image,Class
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [245]:
var, skew, kurt, entropy, label =data['Variance of WTI'], data['Skewness of WTI'], data['Curtosis of WTI'], data['Entropy of image'], data['Class']


1) You should first clean the dataset (handle missing values and  categorical values) and especially : 
- Replace missing values by average or median values
- Center and normalize the data

In [246]:
def count_NaN(data):
    countNaN = np.zeros((np.shape(data)[1]))
    for k in data:
        for i in range (len(data[k])):
            va_k = data[k]
            if va_k[i] == np.NaN:
                countNaN[k] += 1
    return countNaN

count_NaN(data)

#Pas de Nan dans le premier dataset


array([0., 0., 0., 0., 0.])

In [248]:
def replace_NaN(data):
    no_NaN_data = data
    mean_data = np.mean(data)
    count_replaced_values = np.zeros((np.shape(data)[1]))
    for k in data:
        for i in range (len(data[k])):
            va_k = no_NaN_data[k]
            if va_k[i] == np.NaN:
                count_replaced_values[k] += 1
                va_k[i] = mean_data[k] #replace with mean
    return no_NaN_data, count_replaced_values

replace_NaN(data)[0].describe()


    

,Variance of WTI,Skewness of WTI,Curtosis of WTI,Entropy of image,Class
count,1372.000000,1372.000000,1372.000000,1372.000000,1372.000000
mean,0.433735,1.922353,1.397627,-1.191657,0.444606
std,2.842763,5.869047,4.310030,2.101013,0.497103
min,-7.042100,-13.773100,-5.286100,-8.548200,0.000000
25%,-1.773000,-1.708200,-1.574975,-2.413450,0.000000
50%,0.496180,2.319650,0.616630,-0.586650,0.000000
75%,2.821475,6.814625,3.179250,0.394810,1.000000
max,6.824800,12.951600,17.927400,2.449500,1.000000


In [249]:
#In practice we often ignore the shape of the distribution and just transform the data to center it by removing 
# the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

def center_and_normalize(data):
    center_and_normalize = data
    mean_data, std_data = np.mean(data), np.std(data)
    types_data = data.dtypes
    for k in center_and_normalize:
        if types_data[k] == float: #on se souhaite pas centrée réduire les labels
            center_and_normalize[k] = (center_and_normalize[k] - mean_data[k])*(1/std_data[k])
    return center_and_normalize


center_and_normalize(data).describe()



,Variance of WTI,Skewness of WTI,Curtosis of WTI,Entropy of image,Class
count,1.372000e+03,1.372000e+03,1.372000e+03,1.372000e+03,1372.000000
mean,3.678625e-16,-3.509098e-16,-2.531584e-16,-3.680244e-16,0.444606
std,1.000365e+00,1.000365e+00,1.000365e+00,1.000365e+00,0.497103
min,-2.630737e+00,-2.675252e+00,-1.551303e+00,-3.502703e+00,0.000000
25%,-7.765474e-01,-6.188189e-01,-6.899455e-01,-5.817379e-01,0.000000
50%,2.197423e-02,6.771828e-02,-1.812706e-01,2.880644e-01,0.000000
75%,8.402427e-01,8.338757e-01,4.135174e-01,7.553713e-01,1.000000
max,2.249008e+00,1.879908e+00,3.836586e+00,1.733680e+00,1.000000


2. Split the dataset : 
- Split between training set and test set
- Split the training set for cross-validation

In [250]:
clean_data = replace_NaN(data)[0]
clean_data = center_and_normalize(clean_data)

clean_data.describe()


,Variance of WTI,Skewness of WTI,Curtosis of WTI,Entropy of image,Class
count,1.372000e+03,1.372000e+03,1.372000e+03,1.372000e+03,1372.000000
mean,-2.301366e-16,-2.338589e-17,-1.047915e-17,3.042594e-17,0.444606
std,1.000365e+00,1.000365e+00,1.000365e+00,1.000365e+00,0.497103
min,-2.630737e+00,-2.675252e+00,-1.551303e+00,-3.502703e+00,0.000000
25%,-7.765474e-01,-6.188189e-01,-6.899455e-01,-5.817379e-01,0.000000
50%,2.197423e-02,6.771828e-02,-1.812706e-01,2.880644e-01,0.000000
75%,8.402427e-01,8.338757e-01,4.135174e-01,7.553713e-01,1.000000
max,2.249008e+00,1.879908e+00,3.836586e+00,1.733680e+00,1.000000


In [273]:
from sklearn.model_selection import train_test_split

#Faire aussi cross validation

def split_data(data, test_size):
    labels = data.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_size)
    return X_train, X_test, y_train, y_test




In [285]:
# for k in data:
#         shapiro_test = shapiro(data[k])
#         A = shapiro_test.pvalue
# A
# len(data)
data.iloc[:,1]

0       1.149455
1       1.064453
2      -0.777352
3       1.295478
4      -1.087038
          ...   
1367   -0.097693
1368   -1.158984
1369   -2.621646
1370   -1.756471
1371   -0.439822
Name: Skewness of WTI, Length: 1372, dtype: float64

You may implement feature selection: bruteforce, by looking at correlations, from an ACP (for classiffication)

In [288]:
#Assumption to check --> Data are gaussian distributed --> check with Shapiro Test

#En statistique, le test de Shapiro–Wilk teste l'hypothèse nulle selon laquelle un échantillon x 1 , … , x n 
#est issu d'une population normalement distribuée. 

from scipy.stats import shapiro
from sklearn.decomposition import PCA


def shapiro_test(data):
    n = np.shape(data)[1]
    p_values = np.zeros((n))
    for k in range(n):
        data_k = data.iloc[:,k]
        shapiro_test = shapiro(data_k)
        p_values[k] = shapiro_test.pvalue
    return p_values

shapiro_test(clean_data)


def pca(data):
    pca = PCA.fit(data)
    data_reduc = pca.transform(data)
    return data_reduc

pca(clean_data)
    

TypeError: fit() missing 1 required positional argument: 'X'